In [6]:
import urllib2
import urllib
import pandas as pd
from bs4 import BeautifulSoup
from collections import OrderedDict
import time

In [7]:
#specify the url
base_url = 'https://www.naukri.com/c-jobs'

In [10]:

req = urllib.Request(base_url, headers={'User-Agent' : "Magic Browser"}) 
source = urllib.urlopen( req ).read()


AttributeError: 'module' object has no attribute 'Request'

In [5]:
soup= BeautifulSoup(source,'html.parser')  # use lxml to remove the warning which is markuptype

In [6]:
#print soup.prettify()


SyntaxError: invalid syntax (<ipython-input-6-df26e0f5c8a7>, line 1)

In [7]:

#search div tag for class count to get the total number number of jobs  for eg. 1-50 of 44661 and split
#the number of jobs and number of pages from there 
num_jobs = int(soup.find("div", { "class" : "count" }).h1.contents[1].getText().split(' ')[-1])


In [10]:
print ('Total Number of jobs  :{}'.format(num_jobs))

Total Number of jobs  :43924


In [11]:
#extract number of pages as 50 job per page 
import math
#number of pages
num_pages = int(math.ceil(num_jobs/50.0))
#print total number of page
num_pages

879

In [12]:

labels = ['Salary', 'Industry', 'Functional Area', 'Role Category', 'Role','Employment Type']

In [13]:
edu_labels = ['UG', 'PG', 'Doctorate']

In [29]:

naukri_df = pd.DataFrame()#create a dataframe to fill it later

In [30]:
#parse the job data 

def parseJobData(jd_soup):
   
    try:
        #job descripttion
        jd_text = jd_soup.find("ul",{"itemprop":"description"}).getText().strip()
            
        #job location
        location = jd_soup.find("div",{"class":"loc"}).getText().strip()
            
        # Experience
        experience = jd_soup.find("span",{"itemprop":"experienceRequirements"}).getText().strip()
            
        role_info = [content.getText().split(':')[-1].strip() for content in jd_soup.find("div",{"class":"jDisc mt20"}).contents if len(str(content).replace(' ',''))!=0]
        role_info_dict = {label: role_info for label, role_info in zip(labels, role_info)}
            
        #key Skills
        key_skills = '|'.join(jd_soup.find("div",{"class":"ksTags"}).getText().split('  '))[1:]

        #edu_info
        edu_info = [content.getText().split(':') for content in jd_soup.find("div",{"itemprop":"educationRequirements"}).contents if len(str(content).replace(' ',''))!=0]
           
        
       # print(edu_info)
            
        edu_info_dict = {label.strip(): edu_info.strip() for label, edu_info in edu_info}
        for l in edu_labels:
            if l not in edu_info_dict.keys():
                edu_info_dict[l] = ''

        company_name = jd_soup.find("div",{"itemprop":"hiringOrganization"}).contents[1].p.getText().strip()
    except AttributeError:
        return -1
    df_dict = OrderedDict({'Location':location, 'Link':url,'Job Description':jd_text,'Experience':experience,'Skills':key_skills,'Company Name':company_name})
    df_dict.update(role_info_dict)
    df_dict.update(edu_info_dict)  
    time.sleep(1)
    return df_dict
        

#### As of now trying to retrieve the data from one page later increase

In [31]:
#https://www.naukri.com/c-jobs retrieve page wise data
#
for page in range(1,10): #replace  ith numpages+1 
    page_url = base_url+str(page)
    req = urllib.request.Request(page_url, headers={'User-Agent' : "Magic Browser"}) 
    source = urllib.request.urlopen( req ).read()
    soup = BeautifulSoup(source,"html.parser")
    

    #job listing example as all the job listing link start with a and then further prefix by job-listings
    #<a class="content" count="4" href="https://www.naukri.com/job-listings-Network-Engineer-C-c-Civil-Maps-Hyderabad-4-to-9-years-191217901277?
    # src=jobsearchDesk&amp;sid=15137324767867&amp;xp=4&amp;px=1" target="_blank">
    
  
    all_links = [link.get('href') for link in soup.findAll('a') if 'job-listings' in  str(link.get('href'))]
    
    #print all_links
    
    #for all the link retrieve the data 
    
    for url in all_links:
        
        req = urllib.request.Request(url, headers={'User-Agent' : "Magic Browser"})
        
        #job descrption data open the link in the browser
        jd_source = urllib.request.urlopen(req).read()
        
        jd_soup = BeautifulSoup(jd_source,"html.parser")
        
        #parse job data
        if(parseJobData(jd_soup)==-1):
            continue
        else:
            naukri_df = naukri_df.append(parseJobData(jd_soup),ignore_index=True)   
            
      
    
    print(page)

1
2
3
4
5
6
7
8
9


In [32]:
naukri_df.head()

,Company Name,Doctorate,Employment Type,Experience,Functional Area,Industry,Job Description,Link,Location,PG,Role,Role Category,Salary,Skills,UG
0,Sukhvarsha Management Services Pvt. Ltd.,Doctorate Not Required,"Permanent Job, Full Time",2 - 4 yrs,"Medical , Healthcare , R&D , ...",Medical / Healthcare / Hospitals,Experience: 2 + years of Experience mandate C...,https://www.naukri.com/job-listings-Opening-fo...,"Bengaluru, Hyderabad",Any Postgraduate - Any Specialization,Medical Officer,Medical Professional,"INR 3,00,000 - 5,00,000 P.A.",Medical Coder Trainee|Medical Coding Executive...,Any Graduate - Any Specialization
1,Tyfone Communications Development (India) Pvt....,Doctorate Not Required,"Permanent Job, Full Time",7 - 9 yrs,IT Software - Mobile,IT-Software / Software Services,Job Description : 1. Develop and maintain T...,https://www.naukri.com/job-listings-IOS-Develo...,Bengaluru,"M.Tech - Computers, MCA - Computers, MS/M.Sc(S...",Software Developer,Programming & Design,Not Disclosed by Recruiter,Objective C|XCode|IPad|Ios Development|IPhone|...,B.Tech/B.E. - Computers
2,Sukhvarsha Management Services Pvt. Ltd.,Doctorate Not Required,"Permanent Job, Full Time",2 - 4 yrs,"Medical , Healthcare , R&D , ...",Medical / Healthcare / Hospitals,Experience: 2 + years of Experience mandate C...,https://www.naukri.com/job-listings-Opening-fo...,"Hyderabad, Bengaluru",Any Postgraduate - Any Specialization,Medical Officer,Medical Professional,"INR 3,00,000 - 5,00,000 P.A.",Medical Coder Trainee|Medical Coding Executive...,Any Graduate - Any Specialization
3,CREDAS SOLUTIONS PRIVATE LIMITED,Doctorate Not Required,"Permanent Job, Full Time",6 - 11 yrs,"IT Software - Application Programming , ...",IT-Software / Software Services,- We have Job opening for Full Stack Developer...,https://www.naukri.com/job-listings-Full-Stack...,Chennai,Any Postgraduate - Any Specialization,Software Developer,Programming & Design,Not Disclosed by Recruiter,Javascript|Java|Node.Js|Angularjs|C|MongoDB|No...,Any Graduate - Any Specialization
4,Mount Talent Consulting Private Limited,Doctorate Not Required,"Permanent Job, Full Time",5 - 10 yrs,"HR , Recruitment , Administration ...",IT-Software / Software Services,"Dear Folks, We have urgent opening for the ...",https://www.naukri.com/job-listings-Urgent-Ope...,Mumbai,MBA/PGDM - Any Specialization,Pay Roll/Compensation Manager,HR/ Recruitment / IR,Not Disclosed by Recruiter,compensation and benefits|c&b;|compensation|co...,Any Graduate


In [33]:
naukri_df.shape

(402, 15)

In [35]:
#write it into file for further processing

naukri_df.to_csv("Naukri_JD.csv")